In [1]:
from enum import Enum

import httpx
from dotenv import load_dotenv
from markdownify import markdownify as md
from openai import OpenAI
from pydantic import BaseModel
from pydantic import Field
from rich import print

In [2]:
load_dotenv()

True

In [3]:
# binance spot api
url = "https://developers.binance.com/docs/binance-spot-api-docs"
max_length = 5000
model = "gpt-4o-mini"
temperature = 0
max_tokens = 1000

In [4]:
resp = httpx.get(url)
resp.raise_for_status()

print(resp.text[:1000])

<!doctype html>
<html lang="en" dir="ltr" class="docs-wrapper plugin-docs plugin-id-default docs-version-current docs-doc-page 
docs-doc-id-binance-spot-api-docs/CHANGELOG" data-has-hydrated="false">
<head>
<meta charset="UTF-8">
<meta name="generator" content="Docusaurus v3.4.0">
<title data-rh="true">Changelog | Binance Open Platform</title><meta data-rh="true" name="viewport" 
content="width=device-width,initial-scale=1"><meta data-rh="true" name="twitter:card" 
content="summary_large_image"><meta data-rh="true" property="og:url" 
content="https://developers.binance.com/docs/binance-spot-api-docs"><meta data-rh="true" property="og:locale" 
content="en"><meta data-rh="true" property="og:locale:alternate" content="zh_CN"><meta data-rh="true" 
name="docusaurus_locale" content="en"><meta data-rh="true" name="docsearch:language" content="en"><meta 
data-rh="true" name="docusaurus_version" content="current"><meta data-rh="true" name="docusaurus_tag" 
content="docs-default-current"><meta data-rh="

In [5]:
markdown_content = md(resp.content, strip=["a", "img"])
print(markdown_content[:1000])

Changelog \| Binance Open Platform








Skip to main contentProducts▼SearchEnglish* English
* 简体中文
 Spot Trading * Changelog
* Readme
* Filters
* Enums
* Rest API
* Fix API
* Web Socket Streams
* User Data Stream
* Web Socket API
* Testnet
* Error
* FAQs
* 
* Changelog
On this page
CHANGELOG for Binance's API
===========================


**Last Updated: 2025\-01\-09**


### 2025\-01\-09​


* FIX Market Data will be available at **January 16, 05:00 UTC**. The FIX API documentation has been updated 
regarding this feature.
* Please refer to this link for the QuickFix Schema for FIX Market Data.




---


### 2024\-12\-17​


General Changes:


The system now supports microseconds in all related time and/or timestamp fields. Microsecond support is 
**opt\-in**, by default the requests and responses still use milliseconds. Examples in documentation are also using
milliseconds for the foreseeable future.


WebSocket Streams


* A new optional parameter `timeUnit` can be used in the co

## Structured Outputs

- [OpenAI docs](https://platform.openai.com/docs/guides/structured-outputs)
- [OpenAI playground](https://platform.openai.com/playground/chat)


In [6]:
class Category(str, Enum):
    BREAKING_CHANGES = "breaking changes"
    NEW_FEATURES = "new features"
    DEPRECATIONS = "deprecations"
    BUG_FIXES = "bug fixes"
    PERFORMANCE_IMPROVEMENTS = "performance improvements"
    SECURITY_UPDATES = "security updates"


class Date(BaseModel):
    year: int
    month: int
    day: int


class Change(BaseModel):
    content: str
    category: Category


class Entry(BaseModel):
    date: Date
    changes: list[Change] = Field(..., description="The changes made")


class Changelog(BaseModel):
    entries: list[Entry]


print(Changelog.model_json_schema())

{
    '$defs': {
        'Category': {
            'enum': [
                'breaking changes',
                'new features',
                'deprecations',
                'bug fixes',
                'performance improvements',
                'security updates'
            ],
            'title': 'Category',
            'type': 'string'
        },
        'Change': {
            'properties': {
                'content': {'title': 'Content', 'type': 'string'},
                'category': {'$ref': '#/$defs/Category'}
            },
            'required': ['content', 'category'],
            'title': 'Change',
            'type': 'object'
        },
        'Date': {
            'properties': {
                'year': {'title': 'Year', 'type': 'integer'},
                'month': {'title': 'Month', 'type': 'integer'},
                'day': {'title': 'Day', 'type': 'integer'}
            },
            'required': ['year', 'month', 'day'],
            'title': 'Date',
            'type': 'object'
        },
        'Entry': {
            'properties': {
                'date': {'$ref': '#/$defs/Date'},
                'changes': {
                    'description': 'The changes made',
                    'items': {'$ref': '#/$defs/Change'},
                    'title': 'Changes',
                    'type': 'array'
                }
            },
            'required': ['date', 'changes'],
            'title': 'Entry',
            'type': 'object'
        }
    },
    'properties': {'entries': {'items': {'$ref': '#/$defs/Entry'}, 'title': 'Entries', 'type': 'array'}},
    'required': ['entries'],
    'title': 'Changelog',
    'type': 'object'
}

In [7]:
client = OpenAI()


system_prompt = """
Extract information from the provided API changelog and structure it according to the specified schema.
Only use information directly present in the context and do not fabricate or create placeholders.

# Steps
1. **Identify the Date**: Extract the date for each set of changes in the format of year, month, and day.
2. **Categorize Changes**: For each change mentioned, determine its category from the given options:
   - Breaking Changes
   - New Features
   - Deprecations
   - Bug Fixes
   - Performance Improvements
   - Security Updates
3. **Structure the Data**: For each date, compile all the changes and their categories.

# Notes
- Ensure that all extracted information directly corresponds to what is provided in the changelog.
- Avoid assumptions or additions outside the given text.
- Maintain correctness and completeness according to the specified schema.
""".strip()
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": markdown_content[:max_length]},
]

response = client.beta.chat.completions.parse(
    messages=messages,
    model=model,
    temperature=temperature,
    max_tokens=max_tokens,
    response_format=Changelog,
)


parsed = response.choices[0].message.parsed
print(parsed)

Changelog(
    entries=[
        Entry(
            date=Date(year=2025, month=1, day=9),
            changes=[
                Change(
                    content='FIX Market Data will be available at January 16, 05:00 UTC. The FIX API documentation 
has been updated regarding this feature.',
                    category=<Category.NEW_FEATURES: 'new features'>
                )
            ]
        ),
        Entry(
            date=Date(year=2024, month=12, day=17),
            changes=[
                Change(
                    content='The system now supports microseconds in all related time and/or timestamp fields. 
Microsecond support is opt-in, by default the requests and responses still use milliseconds.',
                    category=<Category.NEW_FEATURES: 'new features'>
                ),
                Change(
                    content='A new optional parameter `timeUnit` can be used in the connection URL to select the 
time unit for WebSocket Streams.',
                    category=<Category.NEW_FEATURES: 'new features'>
                ),
                Change(
                    content='A new optional header `X-MBX-TIME-UNIT` can be sent in the request to select the time 
unit for REST API.',
                    category=<Category.NEW_FEATURES: 'new features'>
                ),
                Change(
                    content='Timestamp parameters can now be passed in milliseconds or microseconds for REST API.',
                    category=<Category.NEW_FEATURES: 'new features'>
                ),
                Change(
                    content='A new optional parameter `timeUnit` can be used in the connection URL to select the 
time unit for WebSocket API.',
                    category=<Category.NEW_FEATURES: 'new features'>
                ),
                Change(
                    content='Timestamp parameters can now be passed in milliseconds or microseconds for WebSocket 
API.',
                    category=<Category.NEW_FEATURES: 'new features'>
                ),
                Change(
                    content='A new optional parameter `timeUnit` can be used in the connection URL to select the 
time unit for User Data Streams.',
                    category=<Category.NEW_FEATURES: 'new features'>
                )
            ]
        ),
        Entry(
            date=Date(year=2024, month=12, day=9),
            changes=[
                Change(
                    content='Timestamp parameters now reject values too far into the past or the future.',
                    category=<Category.BREAKING_CHANGES: 'breaking changes'>
                ),
                Change(
                    content='The field for quote order quantity (`origQuoteOrderQty`) has been added to responses 
that previously did not have it.',
                    category=<Category.NEW_FEATURES: 'new features'>
                ),
                Change(
                    content='A new schema 2:1 spot_2_1.xml has been released. The current schema 2:0 spot_2_0.xml 
will thus be deprecated, and retired from the API in 6 months as per our schema deprecation policy.',
                    category=<Category.DEPRECATIONS: 'deprecations'>
                ),
                Change(
                    content='Schema 2:1 is a backward compatible update of schema 2:0.',
                    category=<Category.NEW_FEATURES: 'new features'>
                )
            ]
        )
    ]
)

In [8]:
for change in parsed.entries:
    print(f"Date: {change.date}")

    for item in change.changes:
        print(item)

Date: year=2025 month=1 day=9

Change(
    content='FIX Market Data will be available at January 16, 05:00 UTC. The FIX API documentation has been updated
regarding this feature.',
    category=<Category.NEW_FEATURES: 'new features'>
)

Date: year=2024 month=12 day=17

Change(
    content='The system now supports microseconds in all related time and/or timestamp fields. Microsecond support 
is opt-in, by default the requests and responses still use milliseconds.',
    category=<Category.NEW_FEATURES: 'new features'>
)

Change(
    content='A new optional parameter `timeUnit` can be used in the connection URL to select the time unit for 
WebSocket Streams.',
    category=<Category.NEW_FEATURES: 'new features'>
)

...

Date: year=2024 month=12 day=9

Change(
    content='Timestamp parameters now reject values too far into the past or the future.',
    category=<Category.BREAKING_CHANGES: 'breaking changes'>
)

Change(
    content='The field for quote order quantity (`origQuoteOrderQty`) has been added to responses that previously 
did not have it.',
    category=<Category.NEW_FEATURES: 'new features'>
)

Change(
    content='A new schema 2:1 spot_2_1.xml has been released. The current schema 2:0 spot_2_0.xml will thus be 
deprecated, and retired from the API in 6 months as per our schema deprecation policy.',
    category=<Category.DEPRECATIONS: 'deprecations'>
)

Change(
    content='Schema 2:1 is a backward compatible update of schema 2:0.',
    category=<Category.NEW_FEATURES: 'new features'>
)